In [2]:
# prompt: open this in pandas https://raw.githubusercontent.com/Jackbaude/slipv6/refs/heads/main/pops.csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipaddress import IPv6Network, IPv4Network, IPv4Address

pops_url = "https://raw.githubusercontent.com/Jackbaude/slipv6/refs/heads/main/pops.csv"
geoip_url = "https://raw.githubusercontent.com/Jackbaude/slipv6/refs/heads/main/geoipstarlink.csv"

pops_df = pd.read_csv(pops_url)
geoip_df = pd.read_csv(geoip_url);

ipv4_df = geoip_df[geoip_df['ip_range'].str.contains(r'\.')]
ipv6_df = geoip_df[geoip_df['ip_range'].str.contains(r'\:')]

ipv4_df
ipv6_df

,ip_range,country code,region_code,city,Unnamed: 4
1731,2a0d:3340:f400::/38,DE,DE-BE,Berlin,NaN
1732,2406:2d40:2000::/40,AU,AU-WA,Perth,NaN
1733,2406:2d40:3000::/40,JP,JP-13,Tokyo,NaN
1734,2406:2d40:3300::/40,MN,MN-1,Ulaanbaatar,NaN
1735,2406:2d40:3400::/40,PH,PH-00,Manila,NaN
...,...,...,...,...,...
2213,2605:59c7:f009::/48,US,US-TX,Starbase,NaN
2214,2605:59c7:f080::/48,US,US-FL,Miami,NaN
2215,2605:59c7:f081::/48,US,US-FL,Miami,NaN
2216,2605:59c7:f0c0::/48,US,US-FL,Miami,NaN


In [3]:
def generate_starlink_addresses(prefixes, subprefix_length=56):
    """
    Generate IPv6 addresses for the given prefixes and return them as a list.
    :param prefixes: List of base prefixes (e.g., ['2605:59ca:1300::/40'])
    :param subprefix_length: Subnet size to enumerate (default is /56)
    :return: List of generated IPv6 addresses
    """
    addresses = []
    for base_prefix in prefixes:
        base_network = IPv6Network(base_prefix)
        print(f"Generating addresses for {base_prefix}")
        # Generate all /56 sub-prefixes
        for subprefix in base_network.subnets(new_prefix=subprefix_length):
            # Fix the 57th to 127th bits to 0, 128th bit to 1
            addr = f"{subprefix.network_address}1"
            addresses.append(addr)
    return addresses



In [4]:
# Extract IPv6 prefixes from the DataFrame
ipv6_prefixes = ipv6_df['ip_range'].tolist()[:5]

prefixes = ['2605:59ca:1300::/40', '2605:59ca:1600::/40']

# Generate all possible IPv6 addresses
all_ipv6_addresses = generate_starlink_addresses(prefixes)

# write these ip addrs to a file
with open('addresses.txt', 'w') as f:
    for ip in all_ipv6_addresses:
        f.write(ip + '\n')

Generating addresses for 2605:59ca:1300::/40
Generating addresses for 2605:59ca:1600::/40


In [6]:
# Step 2: Use nmap to scan the addresses
!xmap -v -6 -iL addresses.txt -oN starlink_scan_results.txt

Jan 23 21:07:08.029 [DEBUG] xmap: xmap main thread started
Jan 23 21:07:08.029 [DEBUG] xmap: loaded configuration file /etc/xmap/xmap.conf
Jan 23 21:07:08.029 [DEBUG] xmap: syslog support enabled
Jan 23 21:07:08.029 [DEBUG] xmap: no `-6'|`-4' flag given, default select to send ipv6 packet
Jan 23 21:07:08.029 [DEBUG] xmap: no `-x|--max-len=' given, default select to scan all /32 for ipv6
Jan 23 21:07:08.029 [DEBUG] xmap: no output module provided. will use `csv'.
Jan 23 21:07:08.029 [DEBUG] xmap: no probe module provided. will use `icmp_echo'.
Jan 23 21:07:08.029 [DEBUG] xmap: no iid module provided. will use `low'.
Jan 23 21:07:08.029 [DEBUG] xmap: probe module does not supply application success field.
Jan 23 21:07:08.029 [DEBUG] xmap: requested output field (0): saddr
Jan 23 21:07:08.029 [DEBUG] xmap: no output filter specified. will use default: exclude duplicates and unssuccessful
Jan 23 21:07:08.030 [DEBUG] blocklist: IPv6_MAX_PREFIX_LEN=128
Jan 23 21:07:08.030 [DEBUG] blocklist: 